In [3]:
from datareader.datareader import DatasetReader
from retriever.retriever_topk import TopkRetriever

obj = DatasetReader(embed = True, embedding_model_path='/data/lhb/huggingface/model/tokenizer/Qwen2.5-7B-instruct')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
obj.embedding_with_LM()


119it [02:08,  1.25s/it]

## 读取测试数据

In [3]:
import pickle

with open('/data/lhb/test-openicl-0.1.8/EHR_Base/results/readmission_prediction/randseed42/bge_embed_test42.pkl', 'rb') as file:
    test_embedding_ds = pickle.load(file)
    
test_embedding_ds[0]

{'embed': array([ 0.03450291, -0.0017248 , -0.04685855, ...,  0.01775567,
        -0.03873008, -0.03169431], dtype=float32),
 'id': 0,
 'raw_text': "The patient is Female and was diagnosed with various diseases such as Grand mal status、Pneumonitis due to inhalation of food or vomitus、Acidosis, and in response to these diagnoses, the doctor prescribed medications such as Divalproex Sodium 500 MG Enteric Coated Tablet [Depakote]、phenytoin sodium 100 MG Extended Release Oral Capsule、200 ACTUAT Albuterol 0.09 MG/ACTUAT / Ipratropium Bromide 0.018 MG/ACTUAT Metered Dose Inhaler [Combivent]、topiramate 25 MG Oral Tablet [Topamax]、Amoxicillin 875 MG / Clavulanate 125 MG Oral Tablet、0、heparin sodium, porcine 5000 UNT/ML Injectable Solution、1 ML lorazepam 2 MG/ML Cartridge、5 ML valproic acid 100 MG/ML Injection [Depacon]、folic acid 5 MG/ML Injectable Solution、50 ML sodium chloride 9 MG/ML Injection、1000 ML glucose 50 MG/ML / sodium chloride 4.5 MG/ML Injection、Phenytoin sodium 50 MG/ML Injectabl

In [25]:
import numpy as np

embed = test_embedding_ds[0]['embed']  # 由于 search 的参数需要是 二维的，这里增加一个维度
embed = np.float32(embed)

In [21]:
embed = embed[0].tolist()
embed = np.float32([embed])

In [22]:
embed

array([[ 0.00046571, -0.00426205, -0.00105008, ...,  0.00514582,
         0.00326655, -0.00246059]], dtype=float32)

In [7]:
import faiss

train_index = faiss.read_index('/data/lhb/test-openicl-0.1.8/EHR_Base/faiss_index/faiss_index_qwen7B_train_readmission.index')

In [26]:
train_index.search(embed, 3)

(array([[0.99606556, 0.9957382 , 0.99548733]], dtype=float32),
 array([[22936, 46523, 15185]]))

## random_idx

In [ ]:
import random
random.seed(36)

def generate_random_lists(num_lists):
    """
    生成一个二维列表，每个元素是一个包含3个随机整数的一维列表。
    随机整数范围是0到4000。

    参数:
        num_lists (int): 二维列表中一维列表的数量。

    返回:
        list: 生成的二维列表。
    """
    result = []
    for _ in range(num_lists):
        # 生成一个包含3个随机整数的一维列表
        random_list = [random.randint(0, 47179) for _ in range(3)]
        # 将一维列表添加到二维列表中
        result.append(random_list)
    return result

# 示例用法
num_lists = 200  # 假设我们想要生成5个一维列表
random_2d_list = generate_random_lists(num_lists)
print(random_2d_list)

[[41905, 7296, 1639], [18024, 16049, 14628], [9144, 6717, 44348], [35741, 5697, 38698], [27651, 2082, 1952], [6140, 14328, 15247], [33118, 39453, 1739], [36781, 13031, 46925], [42590, 45962, 35713], [27493, 14446, 29439], [38618, 18231, 425], [10463, 45753, 27696], [22298, 18210, 10189], [14110, 22059, 6698], [6078, 24898, 6338], [23526, 22541, 39565], [17335, 2847, 30108], [35142, 8180, 24807], [5164, 36178, 19213], [41198, 40535, 23700], [37837, 12601, 46174], [4558, 3003, 43336], [14935, 18965, 5229], [15256, 6619, 24911], [18217, 29714, 41660], [23909, 10659, 24260], [23283, 13730, 43920], [17496, 45994, 44796], [42469, 4679, 39920], [41613, 11215, 35005], [16043, 10708, 30294], [24867, 17691, 41943], [45099, 36500, 14392], [44866, 21252, 3665], [15010, 2103, 20673], [26290, 17546, 4337], [13826, 37170, 47049], [20622, 13934, 42954], [32717, 25928, 42129], [30071, 9363, 17359], [9150, 16162, 36789], [35322, 17219, 38311], [28077, 38242, 26175], [23723, 14373, 9065], [33392, 32343, 

In [27]:
import json

with open('rand_idx.json', 'w', encoding='utf-8') as file:
    json.dump(random_2d_list, file, ensure_ascii=False, indent=4)